In [1]:
# Code is not working here - but it works with spark-submit

In [2]:
# Set environment variables
import os
os.environ['AZURE_SPARK'] ='1'
os.environ['SPARK_MAJOR_VERSION'] ='2'
os.environ['HDP_VERSION'] ='2.6.5.8-7'
os.environ['JAVA_HOME'] ='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME'] ='/usr/hdp/current/spark2-client'
os.environ['HADOOP_HOME'] ='/usr/hdp/2.6.5.8-7/hadoop'
os.environ['HADOOP_CONF_DIR'] ='/etc/hadoop/conf'
os.environ['SPARK_CONF_DIR'] ='/usr/hdp/current/spark2-client/conf'
os.environ['PYSPARK_PYTHON'] ='/anaconda/envs/py35/bin/python'
os.environ['SPARK_DIST_CLASSPATH'] =':/usr/hdp/current/spark2-client/jars/*:/usr/lib/hdinsight-datalake/*:/usr/hdp/current/spark_llap/*:/usr/hdp/current/spark2-client/conf:'
os.environ['LD_LIBRARY_PATH'] ='/usr/hdp/current/hadoop-client/lib/native:/usr/hdp/current/hadoop-client/lib/native/Linux-amd64-64'
os.environ['_HADOOP_LZO_JAR'] ='/usr/hdp/2.6.5.8-7/hadoop/lib/hadoop-lzo-0.6.0.2.6.5.8-7.jar'

In [3]:
#Important step - Find the spark configuration
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
        .appName("kirti-test-3core-12g") \
        .master("yarn") \
        .config("fs.azure.account.key.qbsecstorage.blob.core.windows.net", \
                "3CUkUfpScv2RctZKth4t/iZPPk6DJa2935jZtyjpLWJSjnNm8/dDfgWnR+5d0w9RU2AIucoUAiU1ZIFWXzbkVw==") \
        .config("spark.executor.cores", "3") \
        .config("spark.executor.memoryOverhead", "1g") \
        .config("spark.executor.memory", "12g") \
        .config("spark.dynamicAllocation.enabled", "true") \
        .config("spark.shuffle.service.enabled", "true") \
        .getOrCreate()

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [6]:
df = spark.read.parquet("wasb://qbdata@qbsecstorage.blob.core.windows.net/data/03_primary/product_holding/parquet/20180716143800/")

In [7]:
df.printSchema()

root
 |-- mask_custid: long (nullable = true)
 |-- acckey: long (nullable = true)
 |-- REF_DATE: string (nullable = true)
 |-- BUSINESS_DATE: string (nullable = true)
 |-- TCP_SIC92: integer (nullable = true)
 |-- TCP_SIC92_DESC: string (nullable = true)
 |-- REPORT_LINE_L5_CODE: string (nullable = true)
 |-- REPORT_LINE_L6_CODE: string (nullable = true)
 |-- REPORT_LINE_L7_CODE: string (nullable = true)
 |-- REPORT_LINE_L5_NAME: string (nullable = true)
 |-- REPORT_LINE_L6_NAME: string (nullable = true)
 |-- REPORT_LINE_L7_NAME: string (nullable = true)
 |-- FI_OR_MI: string (nullable = true)
 |-- TRAD_TRADE_ID: long (nullable = true)
 |-- BOOK_CODE: string (nullable = true)
 |-- PRS_CODE: string (nullable = true)
 |-- PRS_CODE_DESCRIPTION: string (nullable = true)
 |-- SOURCE_KEY: string (nullable = true)
 |-- WALKER_GL: integer (nullable = true)
 |-- ACBS_LINE_NUMBER: string (nullable = true)
 |-- ACBS_FEE_TYPE: string (nullable = true)
 |-- ACBS_CIS_CODE: string (nullable = true)
 

In [8]:
df2 = df.selectExpr("1 as rn").limit(100)

In [9]:
df.crossJoin(df2).groupBy("REPORT_LINE_L5_CODE", "BUSINESS_AREA").agg(sum("INCOME_BASE"), count(lit(1))).show()

+--------------------+-------------+--------------------+--------+
| REPORT_LINE_L5_CODE|BUSINESS_AREA|    sum(INCOME_BASE)|count(1)|
+--------------------+-------------+--------------------+--------+
|CRP5_A_FEE_LOAN_DRIP|       CPBTOT| 5.802856476000921E9|53350400|
|       CRP5_GRP_ADAM|       CPBTOT|           1.78179E7|   10500|
|      CRP5_OVERDRAFT|        BBTOT|   47772.99999999991| 1187000|
|      CRP5_DEP_NIBBS|    WLGLYNTOT|  1044633.0000000024|  893500|
|CRP5_A_FEE_LOAN_DRIP|        RBSCR|1.3103197999999996E7|   19700|
|    CRP5_DEP_MANAGED|         null|                 0.0|     700|
|    CRP5_DEP_CUR_IBS|         null| 2.073309339999986E8|10305800|
|                null|        RBSCR|   434510.9999998659| 1648400|
|     CRP5_LIBOR_FEES|        RBSCR| 5.103049299999883E7|  441200|
| CRP5_A_FEE_LOAN_DEF|       CPBTOT|      -8.004313797E9| 4521000|
|      CRP5_INT_TRADE|    WLGLYNTOT|            120000.0|    2200|
|    CRP5_LIBOR_LOANS|     CORP&TSY|1.5119781099999997E8|  244

In [9]:
df.count()

82752013